In [1]:
# @title Подключение к диску с данными
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import polars as pl
from datetime import datetime
import os

# Пути
DATA_PATH = "/content/drive/MyDrive/ml-vtb-data-fusion-strazh/data/"
PARTS = [1, 2, 3]

# Загружаем все train части
train_ops = []
for i in PARTS:
    path = f"{DATA_PATH}train_part_{i}.parquet"
    df = pl.scan_parquet(path)
    train_ops.append(df)

train_ops = pl.concat(train_ops, how="vertical").collect(engine="streaming")

# Парсим время
train_ops = train_ops.with_columns(
    pl.col("event_dttm").str.to_datetime("%Y-%m-%d %H:%M:%S")
)

# Загружаем labels
labels = pl.read_parquet(f"{DATA_PATH}train_labels.parquet")

# Мержим (left join) — где нет label → target=0
train_full = train_ops.join(
    labels.select(["event_id", "target"]),
    on="event_id",
    how="left"
).with_columns(
    pl.col("target").fill_null(0).cast(pl.Int32)
)

# Сохраняем (это будет ~85M строк, но parquet сожмёт до 8–12 ГБ)
train_full.write_parquet(f"{DATA_PATH}train_full.parquet", compression="zstd")

print(train_full.shape)
print(train_full["target"].value_counts())